In [ ]:
# Importing the sys module to manipulate the Python runtime environment
import sys

# Adding a specific path to the beginning of the sys.path list
# This path points to a directory containing Python packages
# This is typically done to ensure that the desired version of a package is used
# In this case, the path points to the site-packages directory of the igwn-py39 environment
sys.path = ['/cvmfs/software.igwn.org/conda/envs/igwn-py39/lib/python3.9/site-packages'] + sys.path

In [ ]:
# Importing the pyplot module from the matplotlib library and aliasing it as plt
from matplotlib import pyplot as plt

# Importing the gwpy library, which provides tools for gravitational wave data analysis
import gwpy

# Importing the TimeSeries class from the gwpy.timeseries module
from gwpy.timeseries import TimeSeries

# Importing the NumPy library and aliasing it as np
import numpy as np

# Importing the Pandas library and aliasing it as pd
import pandas as pd

# Importing the seaborn library for statistical data visualization
import seaborn as sns


In [ ]:
# Setting the path to the directory containing the Einstein Telescope Mock Data Challenge 1 (MDC1) data
MDC_PATH = "/cvmfs/et-gw.osgstorage.org/et-gw/PUBLIC/MDC1/data"

# Defining a list of datasets, each identified by a string (E0, E1, ..., C2)
DATASETS = ['E0','E1','E2','E3','C1','C2']

# Creating a dictionary to map each dataset to its corresponding channel
# Each channel is specified as a string with the dataset name followed by ":STRAIN"
CHANNELS = {n : f'{n}:STRAIN' for n in DATASETS}

In [ ]:
# Reading a gravitational wave strain time series from a GWF file using the gwpy library
# The file path is constructed based on the MDC_PATH and dataset ('E1') information
e0 = TimeSeries.read(MDC_PATH+'/E1/E-E1_STRAIN_DATA-1000000000-2048.gwf', 'E1:STRAIN')

# Printing the first 10 elements of the TimeSeries object (time series data)
# The TimeSeries object contains various metadata and information about the strain data
print(e0[1:10])


In [ ]:
# Enabling inline plotting in Jupyter notebooks or IPython environments
%matplotlib inline

# Creating a plot of the gravitational wave strain data using the plot() method of the TimeSeries object
plot = e0.plot()


In [ ]:
# Computing the amplitude spectral density (ASD) of the gravitational wave strain data
# The ASD is calculated using a Fast Fourier Transform (FFT) with a specified length of 4 seconds
# The median method is used to estimate the ASD
asd = e0.asd(fftlength=4, method="median")

# Creating a plot of the computed ASD using the plot() method of the ASD object
plot = asd.plot()

# Displaying the plot while suppressing any warning messages (if any)
plot.show(warn=False)


In [ ]:
# Getting the current Axes instance from the plot
ax = plot.gca()

# Setting the x-axis (frequency) limits of the plot from 1 Hz to 4000 Hz
ax.set_xlim(1, 4000)

# Setting the y-axis (ASD values) limits of the plot from 1e-25 to 1e-23
ax.set_ylim(1e-25, 1e-23)

# Displaying the updated plot with adjusted axis limits
plot

In [ ]:
# Whitening the strain data using a window of 4 seconds and a step of 2 seconds
white = e0.whiten(8, 4)

In [ ]:
%matplotlib inline
plot = white.plot()



In [ ]:
asd = white.asd(fftlength=4, method="median")
plot = asd.plot()
plot.show(warn=False)

In [ ]:
ax = plot.gca()
ax.set_xlim(1, 4000)
ax.set_ylim(1e-2, 1e-2)
plot

In [ ]:
# Plotting Amplitude Spectral Density (ASD)

# Calculate and plot the ASD (Amplitude Spectral Density) of the original 'strain' data
# The 'fftlength=4' argument sets the length of the Fast Fourier Transform (FFT) window to 4 seconds
fig1 = e0.asd(fftlength=4).plot()

# Calculate and plot the ASD of the whitened 'white' data
# The 'fftlength=4' argument sets the length of the FFT window to 4 seconds
fig2 = white.asd(fftlength=4).plot()

# Uncomment the following lines to set custom x-axis and y-axis limits:
# plt.xlim(10, 2000)   # Set custom x-axis limits (frequency range)
# plt.ylim(1e-24, 1e-19)   # Set custom y-axis limits (ASD values)